In [3]:


# Implementation of Amanatides & Woo 3D voxel traversal (Python)
# This code is user-visible and runnable. It implements traversal of unit voxels (integer grid)
# that a line segment between two 3D points passes through.
#
# API:
#   voxel_traversal(p0, p1) -> yields (x,y,z) integer voxel coordinates in order visited
#   voxel_traversal_list(p0, p1) -> returns list of voxels visited
#
# Notes:
# - Voxels are unit cubes [i, i+1) in each axis and indexed by integers using math.floor.
# - The algorithm is safe for segments starting outside/inside, and handles axis-aligned cases.
# - It follows the Amanatides & Woo algorithm described in the paper.
#
# Quick demo at the end shows example usages.

import math
from typing import Tuple, Iterator, List

INF = float('inf')

def sign(x: float) -> int:
    return 1 if x > 0 else (-1 if x < 0 else 0)

def voxel_traversal(p0: Tuple[float,float,float], p1: Tuple[float,float,float]) -> Iterator[Tuple[int,int,int]]:
    """
    Amanatides & Woo voxel traversal for the segment from p0 to p1.
    p0, p1: (x,y,z) floats
    Yields integer voxel coordinates (ix,iy,iz) in the order the segment visits them.
    """
    x0, y0, z0 = p0
    x1, y1, z1 = p1

    # Direction vector of the segment
    dx = x1 - x0
    dy = y1 - y0
    dz = z1 - z0

    # Current voxel (integer coordinates of the unit cube containing p0)
    ix = math.floor(x0)
    iy = math.floor(y0)
    iz = math.floor(z0)

    # Target voxel containing p1
    tx = math.floor(x1)
    ty = math.floor(y1)
    tz = math.floor(z1)

    yield (ix, iy, iz)

    # If the segment is degenerate (point), we're done
    if dx == 0 and dy == 0 and dz == 0:
        return

    # Steps for each axis (+1, -1, or 0)
    step_x = sign(dx)
    step_y = sign(dy)
    step_z = sign(dz)

    # Compute tMax for each axis: the t (0..1) at which the ray crosses the first voxel boundary
    # and tDelta: how much t increases to cross one voxel along that axis.
    # Parameterise point as p(t) = p0 + t*(p1-p0) with t in [0,1].
    def compute_tmax_tdelta(p0_coord, d, i_voxel, step):
        if d == 0:
            return INF, INF
        # next boundary coordinate along axis
        if step > 0:
            next_boundary = i_voxel + 1.0  # crossing to the voxel at i_voxel+1
        else:
            next_boundary = i_voxel * 1.0  # crossing to the voxel at i_voxel (the lower face)
        tmax = (next_boundary - p0_coord) / d
        # tDelta is the parametric distance to cross one full voxel along this axis
        tdelta = 1.0 / abs(d)
        return tmax, tdelta

    tMaxX, tDeltaX = compute_tmax_tdelta(x0, dx, ix, step_x)
    tMaxY, tDeltaY = compute_tmax_tdelta(y0, dy, iy, step_y)
    tMaxZ, tDeltaZ = compute_tmax_tdelta(z0, dz, iz, step_z)

    # We iterate until we've reached the target voxel
    # Use a safety counter to avoid infinite loops in pathological floating-point cases
    safety = 0
    max_iter = 1000000

    while (ix != tx) or (iy != ty) or (iz != tz):
        safety += 1
        if safety > max_iter:
            raise RuntimeError("Exceeded max iterations in voxel traversal (possible numerical issue)")

        # Choose the axis with the smallest tMax to step
        if tMaxX < tMaxY:
            if tMaxX < tMaxZ:
                # step in x
                ix += step_x
                tMaxX += tDeltaX
            else:
                # step in z
                iz += step_z
                tMaxZ += tDeltaZ
        else:
            if tMaxY < tMaxZ:
                # step in y
                iy += step_y
                tMaxY += tDeltaY
            else:
                # step in z
                iz += step_z
                tMaxZ += tDeltaZ

        yield (ix, iy, iz)

def voxel_traversal_list(p0: Tuple[float,float,float], p1: Tuple[float,float,float]) -> List[Tuple[int,int,int]]:
    return list(voxel_traversal(p0, p1))

# ---------------------- Demo / Tests ----------------------
if __name__ == "__main__":
    examples = [
        ((0.2, 0.2, 0.2), (3.8, 2.3, 1.1)),
        ((-0.5, 0.0, 0.0), (2.4, 0.0, 0.0)),   # axis-aligned along +x
        ((2.9, 2.9, 2.9), (0.1, 0.1, 0.1)),    # descending diagonal
        ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),    # single point
        ((0.9, 0.1, 0.1), (0.1, 3.9, 0.1)),    # mostly y movement crossing x boundary
    ]

    for a,b in examples:
        voxels = voxel_traversal_list(a,b)
        print(f"Segment {a} -> {b}\n  Voxels: {voxels}\n")



Segment (0.2, 0.2, 0.2) -> (3.8, 2.3, 1.1)
  Voxels: [(0, 0, 0), (1, 0, 0), (1, 1, 0), (2, 1, 0), (3, 1, 0), (3, 2, 0), (3, 2, 1)]

Segment (-0.5, 0.0, 0.0) -> (2.4, 0.0, 0.0)
  Voxels: [(-1, 0, 0), (0, 0, 0), (1, 0, 0), (2, 0, 0)]

Segment (2.9, 2.9, 2.9) -> (0.1, 0.1, 0.1)
  Voxels: [(2, 2, 2), (2, 2, 1), (2, 1, 1), (1, 1, 1), (1, 1, 0), (1, 0, 0), (0, 0, 0)]

Segment (0.5, 0.5, 0.5) -> (0.5, 0.5, 0.5)
  Voxels: [(0, 0, 0)]

Segment (0.9, 0.1, 0.1) -> (0.1, 3.9, 0.1)
  Voxels: [(0, 0, 0), (0, 1, 0), (0, 2, 0), (0, 3, 0)]

